해당 문서에 있는 내용
- 1. 3번 스크립트 수집한 리플 데이터 품질을 확인하고
- 2. 데이터 Join해서 1분봉 종가로 1분단위 김프데이터 만들기
- 3. 종가기준 1분단위 김프 데이터로 1시간단위 캔들 만들기

In [1]:
import pandas as pd

In [72]:
XRP_KRW = pd.read_csv('XRP-Upbit-KRW_1min.csv')
XRP_USDT = pd.read_csv('XRP-Binance-USDT_1min.csv')

In [73]:
# index로 넣어주고 정렬
XRP_KRW = XRP_KRW.set_index(['date']).sort_index()
XRP_USDT = XRP_USDT.set_index(['date']).sort_index()

In [74]:
def make_fulltime_data(df, freq='min'):
    df.index = pd.DatetimeIndex(df.index)
    full_idx = pd.date_range(df.index[0],df.index[-1], freq=freq)
    df = df.reindex(full_idx).fillna(method='pad')
    return df

XRP_KRW = make_fulltime_data(XRP_KRW)
XRP_USDT = make_fulltime_data(XRP_USDT)
XRP_KRW_close = XRP_KRW.drop(['open','high','low','volume'],axis=1).rename(columns={'close':'KRW_close'})
XRP_USDT_close = XRP_USDT.drop(['open','high','low','volume'],axis=1).rename(columns={'close':'USDT_close'})

In [104]:
kimp_dat = pd.merge(XRP_KRW_close, XRP_USDT_close,
    left_index=True, right_index=True,
    how = 'inner')
kimp_dat['kimp'] = (kimp_dat['KRW_close'] - kimp_dat['USDT_close']) / kimp_dat['USDT_close'] * 100
kimp_dat.to_csv('kimp_dat_1min.csv', index_label='date')

In [105]:
kimp_dat

,KRW_close,USDT_close,kimp
2018-05-04 17:12:00,973.0,966.699000,0.651806
2018-05-04 17:13:00,993.0,977.440100,1.591903
2018-05-04 17:14:00,1000.0,998.911559,0.108963
2018-05-04 17:15:00,992.0,977.654922,1.467295
2018-05-04 17:16:00,994.0,998.675255,-0.468146
...,...,...,...
2021-11-25 20:30:00,1300.0,1235.862144,5.189726
2021-11-25 20:31:00,1300.0,1235.267064,5.240400
2021-11-25 20:32:00,1300.0,1235.505096,5.220124
2021-11-25 20:33:00,1305.0,1235.743128,5.604472


In [101]:
kimp_dat.loc[(kimp_dat.index>='2021-11-24') & (kimp_dat.index<'2021-11-25')].min()

KRW_close     1270.000000
USDT_close    1200.301069
kimp             4.119331
dtype: float64